In [36]:
import numpy as np
from math import floor, ceil

def findAvg(block):
    return np.mean(block)

def createBinaryPattern(block, avg):
    binaryArr = []
    for i in range(len(block)):
        for j in range(len(block[i])):
            if block[i][j] <= avg:
                binaryArr.append(1)
            else:
                binaryArr.append(0)
    return binaryArr

def calXorConsecutive(binaryArr):
    return [binaryArr[i] ^ binaryArr[i+1] for i in range(0, len(binaryArr), 2)]

def calXor(xorArr, secret_bit_stream):
    return [xorArr[i] ^ secret_bit_stream[i] for i in range(0, len(xorArr))]

# suffle two consecutive places of arr like index 0 and 1, index 2, 3
def suffle(arr):
    for i in range(0, len(arr), 2):
        arr[i], arr[i+1] = arr[i+1], arr[i]
    return arr

def embed(block, secret):
    blockArr = block.flatten()
    idx = 0
    stego_block = []
    for i in range(0, len(blockArr), 2):
        d = abs(blockArr[i]-blockArr[i+1])
        v = floor((blockArr[i]+blockArr[i+1])/2)
        d_dash = d*2 + secret[idx]
        idx += 1
        if blockArr[i] < blockArr[i+1]:
            stego_block.append(v - floor(d_dash/2))
            stego_block.append(v + ceil(d_dash/2))
        else:
            stego_block.append(v + ceil(d_dash/2))
            stego_block.append(v - floor(d_dash/2))
    return np.array(stego_block).reshape(block.shape)

def extract(stego_block):
    blockArr = stego_block.flatten()
    embeded_bits = []
    org_block = []
    for i in range(0, len(blockArr), 2):
        d_dash = abs(blockArr[i]-blockArr[i+1])
        d = floor(d_dash/2)
        v = floor((blockArr[i]+blockArr[i+1])/2)
        if blockArr[i] < blockArr[i+1]:
            org_block.append(v - floor(d/2))
            org_block.append(v + ceil(d/2))
        else:
            org_block.append(v + ceil(d/2))
            org_block.append(v - floor(d/2))
        binary_str = bin(int(d_dash))[2:]
        print(binary_str[-1])
        embeded_bits.append(int(binary_str[-1]))

    return np.array(org_block).reshape(stego_block.shape), embeded_bits


block = np.array([[100, 110, 120, 130],[140, 150, 160, 170],[180, 190, 200, 210],[220, 230, 240, 250]])
secret_bit_stream = np.array([0,1,0,0,0,0,0,1])

avg = findAvg(block)
binaryArr = createBinaryPattern(block, avg)


print(binaryArr)
xorArr = calXorConsecutive(binaryArr)
print(xorArr)
xorArr1 = calXor(xorArr, secret_bit_stream)
print(xorArr1)

suffledArr = suffle(xorArr1)
print(suffledArr)
stego_block = embed(block, suffledArr)
print(stego_block)
org_block, embeded_bits = extract(stego_block)



print(org_block)
print(embeded_bits)
suffledArr_new = suffle(embeded_bits)
print(suffledArr_new)
avg_new = findAvg(org_block)
binaryArr_new = createBinaryPattern(org_block, avg_new)
xorArr_new = calXorConsecutive(binaryArr_new)
xorArr1_new = calXor(xorArr_new, suffledArr_new)
print(xorArr1_new)

[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 1]
[1, 0, 0, 0, 0, 0, 1, 0]
[[ 95 116 115 135]
 [135 155 155 175]
 [175 195 195 215]
 [215 236 235 255]]
1
0
0
0
0
0
1
0
[[100 110 120 130]
 [140 150 160 170]
 [180 190 200 210]
 [220 230 240 250]]
[1, 0, 0, 0, 0, 0, 1, 0]
[0, 1, 0, 0, 0, 0, 0, 1]
[0, 1, 0, 0, 0, 0, 0, 1]


In [85]:
import numpy as np
from math import floor, ceil
import cv2
import random

def generate_binary_bit_stream(length):
  # Generate a binary bit stream of the specified length
  bit_stream = ''.join(random.choice(['0', '1']) for _ in range(length))
  bit_stream_arr = np.array([int(bit) for bit in bit_stream])
  return bit_stream_arr

def findAvg(block):
    return floor(np.mean(block))

def createBinaryPattern(block, avg):
    binaryArr = []
    for i in range(len(block)):
            if block[i] <= avg:
                binaryArr.append(1)
            else:
                binaryArr.append(0)
    return binaryArr

def calXorConsecutive(binaryArr):
    return [binaryArr[i] ^ binaryArr[i+1] for i in range(0, len(binaryArr), 2)]

def calXor(xorArr, secret_bit_stream, idx):
  arr = []
  for i in range(0, len(xorArr)):
    arr.append(xorArr[i] ^ secret_bit_stream[idx])
    idx += 1
  return arr, idx

def xor_arrays(arr1, arr2):
    arr = []
    for i in range(len(arr1)):
        arr.append(arr1[i] ^ arr2[i])

    return arr

# suffle two consecutive places of arr like index 0 and 1, index 2, 3
def suffle(arr):
    for i in range(0, len(arr), 2):
        arr[i], arr[i+1] = arr[i+1], arr[i]
    return arr

def embed(cover_image, secret, B=4):
  H, W = cover_image.shape
  num_blocks_H = H // B
  num_blocks_W = W // B
  blocks = []

  stego_image = cover_image.copy()
  total_embeded_bits = []

  for i in range(num_blocks_H):
    for j in range(num_blocks_W):
      block = cover_image[i*B:(i+1)*B, j*B:(j+1)*B]
      blocks.append(block)

  print('total number of blocks', len(blocks))
  print('First block: \n', blocks[0])
  idx = 0
  for j in range(len(blocks)):
    block = blocks[j].flatten().astype(np.int16)
    stego_block = []
    avg = findAvg(block)
    # print('avg', avg)
    binaryArr = createBinaryPattern(block, avg)
    # print('binaryArr', binaryArr)
    xorArr = calXorConsecutive(binaryArr)
    # print('xorArr', xorArr)
    xorArr1, idx = calXor(xorArr, secret, idx)
    # print('xorArr1', xorArr1)
    suffledArr = suffle(xorArr1)
    # print('suffledArr', suffledArr)
    total_embeded_bits.extend(suffledArr)
    k = 0
    for i in range(0, len(block), 2):
      d = abs(block[i]-block[i+1])
      v = floor((block[i]+block[i+1])/2)
      # print('v',v)
      d_dash = d*2 + suffledArr[k]
      # print('d_dash',d_dash)
      k += 1
      if block[i] < block[i+1]:
          stego_block.append(v - floor(d_dash/2))
          stego_block.append(v + ceil(d_dash/2))
      else:
          stego_block.append(v + ceil(d_dash/2))
          stego_block.append(v - floor(d_dash/2))

    block_row = j // num_blocks_W
    block_col = j % num_blocks_W
    stego_image[block_row*B:(block_row+1)*B, block_col*B:(block_col+1)*B] = np.array(stego_block).reshape(B, B)

  print('total embeded bits', len(total_embeded_bits))
  print('embed bits', total_embeded_bits)

  return stego_image


def extract(stego_image, B=4):
  H, W = stego_image.shape
  num_blocks_H = H // B
  num_blocks_W = W // B
  blocks = []
  extracted_bits = []
  org_image = []

  for i in range(num_blocks_H):
    for j in range(num_blocks_W):
      block = stego_image[i*B:(i+1)*B, j*B:(j+1)*B]
      blocks.append(block)

  # print(blocks)

  print('First block: \n', blocks[0])

  for j in range(len(blocks)):
    extracted_bits_in_a_block = []
    block = blocks[j].flatten().astype(np.int16)
    org_block = []


    for i in range(0, len(block), 2):
      d_dash = abs(block[i]-block[i+1])
      d = floor(d_dash/2)
      v = floor((block[i]+block[i+1])/2)
      if block[i] < block[i+1]:
          org_block.append(v - floor(d/2))
          org_block.append(v + ceil(d/2))
      else:
          org_block.append(v + ceil(d/2))
          org_block.append(v - floor(d/2))
      binary_str = bin(int(d_dash))[2:]
      extracted_bits_in_a_block.append(int(binary_str[-1]))

    block_row = j // num_blocks_W
    block_col = j % num_blocks_W
    # org_image[block_row*B:(block_row+1)*B, block_col*B:(block_col+1)*B] = np.array(org_block).reshape(B, B)
    # print('extracted_bits_in_a_block', extracted_bits_in_a_block)
    suffledArr_new = suffle(extracted_bits_in_a_block)
    # print('suffledArr_new', suffledArr_new)
    avg_new = findAvg(org_block)
    # print('avg_new', avg_new)
    binaryArr_new = createBinaryPattern(org_block, avg_new)
    # print('binaryArr_new', binaryArr_new)
    xorArr_new = calXorConsecutive(binaryArr_new)
    # print('xorArr_new', xorArr_new)
    xorArr1_new = xor_arrays(np.array(xorArr_new), np.array(suffledArr_new))
    # print('xorArr1_new', xorArr1_new)
    extracted_bits.extend(xorArr1_new)

  return extracted_bits



cover_image = cv2.imread('baboon1_4x4.png', cv2.IMREAD_GRAYSCALE)
# cover_image = cv2.imread('baboon.tif', cv2.IMREAD_GRAYSCALE)
print('cover image\n',cover_image)
secret_data = generate_binary_bit_stream(131074)
# secret_data = np.array([0,1,0,1,1,1,1,0])

print('secret data\n', secret_data[:32])

stego_image = embed(cover_image, secret_data)
cv2.imwrite('stego_image.tif', stego_image)
print('stego image\n', stego_image)

print('-----------------------EXTRACTION--------------------------------')

extracted_bits = extract(stego_image)
print('extracted bits\n', extracted_bits)
# cv2.imwrite('original_image.tif', org_image)

if np.array_equal(extracted_bits, secret_data[:len(extracted_bits)]):
  print("Extraction successful!")
else:
  print("Extraction failed.")

cover image
 [[142  62  56  93]
 [115 110  53  68]
 [ 85 108  57  56]
 [ 75 138  97  60]]
secret data
 [0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 1 1 0 1 0 1 1 1 0 0 1 1 0 1 1 1]
total number of blocks 1
First block: 
 [[142  62  56  93]
 [115 110  53  68]
 [ 85 108  57  56]
 [ 75 138  97  60]]
total embeded bits 8
embed bits [1, 1, 1, 1, 1, 1, 1, 1]
stego image
 [[183  22  37 112]
 [118 107  45  76]
 [ 73 120  58  55]
 [ 43 170 116  41]]
-----------------------EXTRACTION--------------------------------
First block: 
 [[183  22  37 112]
 [118 107  45  76]
 [ 73 120  58  55]
 [ 43 170 116  41]]
extracted bits
 [0, 0, 1, 1, 0, 1, 0, 0]
Extraction successful!


In [86]:
import cv2
import numpy as np

def calculate_mse(image1, image2):
    assert image1.shape == image2.shape, "Images must have the same dimensions"

    mse = np.mean((image1 - image2) ** 2)
    return mse

def calculate_psnr(mse, max_pixel_value=255.0):
    if mse == 0:
        return float('inf')  # PSNR is infinite if MSE is zero (identical images)
    psnr = 20 * np.log10(max_pixel_value / np.sqrt(mse))
    return psnr

def load_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    return image

cover_image_path = 'baboon1_4x4.png'
stego_image_path = 'stego_image.tif'

cover_image = load_image(cover_image_path)
stego_image = load_image(stego_image_path)

# Calculate MSE
mse = calculate_mse(cover_image, stego_image)
print(f"MSE: {mse}")

# Calculate PSNR
psnr = calculate_psnr(mse)
print(f"PSNR: {psnr} dB")


MSE: 66.5625
PSNR: 29.898507357490786 dB
